# Lesson 3: Chatbot Example

In this lesson, you will familiarize yourself with the chatbot example you will work on during this course. The example includes the tool definitions and execution, as well as the chatbot code. Make sure to interact with the chatbot at the end of this notebook.

## Import Libraries

In [1]:
import arxiv
import json
import os
from typing import List
from dotenv import load_dotenv
import anthropic

## Tool Functions

In [2]:
PAPER_DIR = "papers"

The first tool searches for relevant arXiv papers based on a topic and stores the papers' info in a JSON file (title, authors, summary, paper url and the publication date). The JSON files are organized by topics in the `papers` directory. The tool does not download the papers.  

In [3]:
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    Search for papers on arXiv based on a topic and store their information.
    
    Args:
        topic: The topic to search for
        max_results: Maximum number of results to retrieve (default: 5)
        
    Returns:
        List of paper IDs found in the search
    """
    
    # Use arxiv to find the papers 
    client = arxiv.Client()

    # Search for the most relevant articles matching the queried topic
    search = arxiv.Search(
        query = topic,
        max_results = max_results,
        sort_by = arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)
    
    # Create directory for this topic
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)
    
    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # Process each paper and add to papers_info  
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info
    
    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)
    
    print(f"Results are saved in: {file_path}")
    
    return paper_ids

In [4]:
search_papers("computers ai browser")

Results are saved in: papers/computers_ai_browser/papers_info.json


['2503.16586v2',
 '1802.03707v1',
 '2111.01677v1',
 '2205.15686v2',
 '2009.03740v1']

The second tool looks for information about a specific paper across all topic directories inside the `papers` directory.

In [5]:
def extract_info(paper_id: str) -> str:
    """
    Search for information about a specific paper across all topic directories.
    
    Args:
        paper_id: The ID of the paper to look for
        
    Returns:
        JSON string with paper information if found, error message if not found
    """
 
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue
    
    return f"There's no saved information related to paper {paper_id}."

In [7]:
print(extract_info('2503.16586v2'))

{
  "title": "Big Help or Big Brother? Auditing Tracking, Profiling, and Personalization in Generative AI Assistants",
  "authors": [
    "Yash Vekaria",
    "Aurelio Loris Canino",
    "Jonathan Levitsky",
    "Alex Ciechonski",
    "Patricia Callejo",
    "Anna Maria Mandalari",
    "Zubair Shafiq"
  ],
  "summary": "Generative AI (GenAI) browser assistants integrate powerful capabilities of\nGenAI in web browsers to provide rich experiences such as question answering,\ncontent summarization, and agentic navigation. These assistants, available\ntoday as browser extensions, can not only track detailed browsing activity such\nas search and click data, but can also autonomously perform tasks such as\nfilling forms, raising significant privacy concerns. It is crucial to\nunderstand the design and operation of GenAI browser extensions, including how\nthey collect, store, process, and share user data. To this end, we study their\nability to profile users and personalize their responses bas

## Tool Schema

Here are the schema of each tool which you will provide to the LLM.

In [12]:
tools = [
    {
        "name": "search_papers",
        "description": "Search for papers on arXiv based on a topic and store their information.",
        "input_schema": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The topic to search for"
                }, 
                "max_results": {
                    "type": "integer",
                    "description": "Maximum number of results to retrieve",
                    "default": 5
                }
            },
            "required": ["topic"]
        }
    },
    {
        "name": "extract_info",
        "description": "Search for information about a specific paper across all topic directories.",
        "input_schema": {
            "type": "object",
            "properties": {
                "paper_id": {
                    "type": "string",
                    "description": "The ID of the paper to look for"
                }
            },
            "required": ["paper_id"]
        }
    }
]

## Tool Mapping

This code handles tool mapping and execution.

In [13]:
mapping_tool_function = {
    "search_papers": search_papers,
    "extract_info": extract_info
}

def execute_tool(tool_name, tool_args):
    
    result = mapping_tool_function[tool_name](**tool_args)

    if result is None:
        result = "The operation completed but didn't return any results."
        
    elif isinstance(result, list):
        result = ', '.join(result)
        
    elif isinstance(result, dict):
        # Convert dictionaries to formatted JSON strings
        result = json.dumps(result, indent=2)
    
    else:
        # For any other type, convert using str()
        result = str(result)
    return result

## Chatbot Code

The chatbot handles the user's queries one by one, but it does not persist memory across the queries.

In [25]:
load_dotenv() 
client = anthropic.Anthropic()

In [26]:
query = "how ai will run the internet ?"
messages = [{'role': 'user', 'content': query}]
    
response = client.messages.create(max_tokens = 2024,
                                model = 'claude-3-7-sonnet-20250219', 
                                messages = messages)

print(response.content[0].text)

# How AI Might Transform Internet Operations

AI is increasingly influencing how the internet functions in several key ways:

## Current and Emerging Roles

- **Content Moderation**: AI systems filter harmful content at scale across platforms
- **Traffic Management**: AI optimizes network routing and bandwidth allocation
- **Personalization**: AI powers recommendation systems and tailored user experiences
- **Cybersecurity**: AI detects and responds to threats in real-time

## Future Possibilities

- **Autonomous Infrastructure**: Self-healing networks that predict and prevent outages
- **Advanced Search**: Moving beyond keywords to understanding context and intent
- **Decentralized Governance**: AI could help manage distributed systems and protocols
- **Intelligent Interfaces**: More natural interaction through conversational AI

However, this transformation raises important considerations around algorithmic bias, privacy, accountability, and human oversight. The most likely future in

In [30]:
response.content

[TextBlock(citations=None, text='# How AI Might Transform Internet Operations\n\nAI is increasingly influencing how the internet functions in several key ways:\n\n## Current and Emerging Roles\n\n- **Content Moderation**: AI systems filter harmful content at scale across platforms\n- **Traffic Management**: AI optimizes network routing and bandwidth allocation\n- **Personalization**: AI powers recommendation systems and tailored user experiences\n- **Cybersecurity**: AI detects and responds to threats in real-time\n\n## Future Possibilities\n\n- **Autonomous Infrastructure**: Self-healing networks that predict and prevent outages\n- **Advanced Search**: Moving beyond keywords to understanding context and intent\n- **Decentralized Governance**: AI could help manage distributed systems and protocols\n- **Intelligent Interfaces**: More natural interaction through conversational AI\n\nHowever, this transformation raises important considerations around algorithmic bias, privacy, accountabil

In [32]:
query =  "diffusion models in NP problem solving"
messages = [{'role': 'user', 'content': query}]
    
response = client.messages.create(max_tokens = 2024,
                                model = 'claude-3-7-sonnet-20250219', 
                                tools = tools,
                                messages = messages)

In [34]:
response.content

[TextBlock(citations=None, text="I'll search for papers related to diffusion models in NP problem solving for you. Let me do that search now.", type='text'),
 ToolUseBlock(id='toolu_01DWHTJTf5d3bwuGg54JLioh', input={'topic': 'diffusion models NP problem solving', 'max_results': 5}, name='search_papers', type='tool_use')]

### Query Processing

In [27]:
def process_query(query):
    
    messages = [{'role': 'user', 'content': query}]
    
    response = client.messages.create(max_tokens = 2024,
                                  model = 'claude-3-7-sonnet-20250219', 
                                  tools = tools,
                                  messages = messages)
    
    running = True
    loop_counter = 1
    while running:
        assistant_content = []
        print(f"round {loop_counter}, content len {len(response.content)}")
        for content in response.content:
            if content.type == 'text':
                
                print(content.text)
                assistant_content.append(content)
                
                if len(response.content) == 1:
                    running = False
            
            elif content.type == 'tool_use':
                
                assistant_content.append(content)
                messages.append({'role': 'assistant', 'content': assistant_content})
                
                tool_id = content.id
                tool_args = content.input
                tool_name = content.name
                print(f"Calling tool {tool_name} with args {tool_args}")
                
                result = execute_tool(tool_name, tool_args)
                messages.append({"role": "user", 
                                  "content": [
                                      {
                                          "type": "tool_result",
                                          "tool_use_id": tool_id,
                                          "content": result
                                      }
                                  ]
                                })
                response = client.messages.create(max_tokens = 2024,
                                  model = 'claude-3-7-sonnet-20250219', 
                                  tools = tools,
                                  messages = messages) 
                
                if len(response.content) == 1 and response.content[0].type == "text":
                    print(response.content[0].text)
                    running = False
        loop_counter+=1

### Chat Loop

In [28]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("\nQuery: ").strip()
            if query.lower() == 'quit':
                break
    
            process_query(query)
            print("\n")
        except Exception as e:
            print(f"\nError: {str(e)}")

Feel free to interact with the chatbot. Here's an example query: 

- Search for 2 papers on "LLM interpretability"

To access the `papers` folder: 1) click on the `File` option on the top menu of the notebook and 2) click on `Open` and then 3) click on `L3`.

In [29]:
chat_loop()

Type your queries or 'quit' to exit.
round 1, content len 2
I'd like to search for papers on arXiv that discuss generative AI as a tool for solving NP problems. Let me do that for you.
Calling tool search_papers with args {'topic': 'generative AI NP problem solver'}
Results are saved in: papers/generative_ai_np_problem_solver/papers_info.json
round 2, content len 2
I've found several papers that might be relevant to generative AI as an NP problem solver. Let me get more detailed information about each of these papers to better understand how they relate to your topic.
Calling tool extract_info with args {'paper_id': '2502.03669v1'}
round 3, content len 1
Calling tool extract_info with args {'paper_id': '0804.4457v1'}
round 4, content len 1
Calling tool extract_info with args {'paper_id': '2504.06126v1'}
round 5, content len 1
Calling tool extract_info with args {'paper_id': '1011.5447v1'}
round 6, content len 1
Calling tool extract_info with args {'paper_id': '2309.03924v1'}
Based on m

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b> To Access the <code>requirements.txt</code> file or the <code>papers</code> folder: </b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em> and finally 3) click on <em>"L3"</em>.
</div>

In the next lessons, you will take out the tool definitions to wrap them in an MCP server. Then you will create an MCP client inside the chatbot to make the chatbot MCP compatible.  

## Resources

[Guide on how to implement tool use](https://docs.anthropic.com/en/docs/build-with-claude/tool-use/overview#how-to-implement-tool-use)

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">


<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

</div>